In [3]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.trial import TrialState

from tqdm.notebook import tqdm

In [2]:
class Config:
    device = torch.device('cpu')
    batch_size = 128
    classes = 10
    wdir = os.getcwd()
    epochs = 10
    log_interval = 10
    n_train_examples = batch_size * 30
    n_valid_examples = batch_size * 10

In [4]:
def get_mnist():
    train_loader = DataLoader(
        datasets.FashionMNIST(
            Config.wdir,
            train=True,
            download=True,
            transform=transforms.ToTensor(),
        ),
        batch_size=Config.batch_size,
        shuffle=True,
    )
    
    valid_loader = DataLoader(
        datasets.FashionMNIST(
            Config.wdir,
            train=False,
            transform=transforms.ToTensor(),
        ),
        batch_size=Config.batch_size,
        shuffle=True,
    )
    
    return train_loader, valid_loader
    

In [9]:
def define_model(trial: optuna.Trial) -> nn.Module:
    n_layers = trial.suggest_int('n_layers', 1, 3)
    layers = []
    
    in_features = 28 * 28
    
    for i in range(n_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float(f'dropout_l{i}', 0.2, 0.5)
        layers.append(nn.Dropout(p))
        
        in_features = out_features
        
    layers.append(nn.Linear(in_features, Config.classes))
    layers.append(nn.LogSoftmax(dim=1))
    
    return nn.Sequential(*layers)    

In [13]:
def objective(trial: optuna.Trial):
    model = define_model(trial).to(Config.device)

    optimizer_name = trial.suggest_categorical(
        'optimizer',
        ['Adam', 'RMSprop', 'SGD'],
    )
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = get_mnist()

    for epoch in range(Config.epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            if batch_idx * Config.batch_size >= Config.n_train_examples:
                break

            data, target = data.view(data.size(0), -1).to(Config.device), target.to(Config.device)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(tqdm(valid_loader)):
                if batch_idx * Config.batch_size >= Config.n_valid_examples:
                    break
                data, target = data.view(data.size(0), -1).to(Config.device), target.to(Config.device)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), Config.n_valid_examples)
        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=500)

[I 2021-05-02 15:51:22,464] A new study created in memory with name: no-name-c1d88b2e-2b2f-4173-be9d-b46739315eaf


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

[I 2021-05-02 15:54:16,033] Trial 98 finished with value: 0.81796875 and parameters: {'n_layers': 1, 'n_units_l0': 54, 'dropout_l0': 0.25166572667924664, 'optimizer': 'RMSprop', 'lr': 0.0025820245170245478}. Best is trial 62 with value: 0.83671875.


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

[I 2021-05-02 15:54:16,325] Trial 99 pruned. 


In [15]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [16]:
print('Study statistics:')
print(f'  Number of finished trials: {len(study.trials)}')
print(f'  Number of pruned trials: {len(pruned_trials)}')
print(f'  Number of complete trials: {len(complete_trials)}')

Study statistics:
  Number of finished trials: 100
  Number of pruned trials: 47
  Number of complete trials: 53


In [17]:
print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

Best trial:
  Value: 0.83671875
  params:
    n_layers: 1
    n_units_l0: 85
    dropout_l0: 0.2435563112434942
    optimizer: RMSprop
    lr: 0.0028943841154470923
